
One of the essential configruations that is needed for model upgrade is the ability to connect to both `registry_uri` and `tracking_uri` across workspaces.

This discovery notebook is deticated to making sure that the registry configurations can be argumented succinctly and therefore this will test the connection to another ws via conventions documented in [Share models across workspaces](https://docs.databricks.com/en/machine-learning/manage-model-lifecycle/multiple-workspaces.html#share-models-across-workspaces).


First, we'll need to inspect some mlflow results to confirm what that format of the uri takes. This is important because we'd like to have any source entity deterministically derive the source uri configurations.

In [0]:
scope = "uc_ml_migrate"
prefix = "genai"

print("genai-host: " + dbutils.secrets.get(scope,f"{prefix}-host"))
print("genai-token: " + dbutils.secrets.get(scope,f"{prefix}-token"))
print("genai-workspace-id: " + dbutils.secrets.get(scope,f"{prefix}-workspace-id"))

In [0]:
import mlflow
from mlflow.tracking import MlflowClient

tracking_uri = 'databricks'
registry_uri = f'databricks://{scope}:{prefix}'

model_name = "vanilla"

# Explicitly set the model registry to the WS model registry, ie NOT "datarbricks-uc"
client = MlflowClient(tracking_uri=tracking_uri,
                      registry_uri=registry_uri)

# Test that we can search a remote workspace:
model_versions = client.search_model_versions(filter_string=f"name='{model_name}'")
print(model_versions)

In [0]:
model_name="vanilla"
experiment_name = f"/Workspace/experiments/uc_ml_migrate/{model_name}"

try:
    experiment_id = client.create_experiment(name=experiment_name[10:])
    experiment = client.get_experiment(experiment_id=experiment_id)
except:
    experiment = client.search_experiments(filter_string=f'name="{experiment_name[10:]}"')[0]
    experiment_id = experiment.experiment_id

In [0]:
mlflow.set_registry_uri("databricks")
registered_model_name = model_name
try:
    client.create_registered_model(name=registered_model_name)
except:
    registered_model_name = client.search_registered_models(filter_string=f'name="{registered_model_name}"')[0].name

In [0]:
def exp_runs():
    return client.search_runs(experiment_ids=[experiment_id])

def rm_versions():
    return client.search_model_versions(filter_string=f"registered_model_name='{registered_model_name}'")

if len(rm_versions())==0:
    registered_model_versions_meta = [{"name": model_name,
                                       "source": f"{r.info.artifact_uri}/{model_name}",
                                       "run_id": r.info.run_id,
                                       "tags": {f"rm_version_tag_{i}": i for i in range(3)}} for i, r in enumerate(exp_runs())]
    for rmv in registered_model_versions_meta:
        client.create_model_version(**rmv)
    for rmv in rm_versions():
        if rmv.version == '3':
            client.transition_model_version_stage(name=registered_model_name, version=3, stage="Production")
        elif rmv.version == '4':
            client.transition_model_version_stage(name=registered_model_name, version=4, stage="Staging")
        elif rmv.__dict__.get('current_stage', 'None') != 'None':
            client.transition_model_version_stage(name=registered_model_name, version=4, stage="None")

In [0]:
model_versions = client.search_model_versions(filter_string=f"name='{model_name}'")
model_versions[0].__dict__

In [0]:
model_versions = client.search_model_versions(filter_string=f"run_id='fb1dffb632aa4d0180f3e00abd1ea51f'")
model_versions[0].__dict__